

---



In [2]:
# Installing the dependencies

!pip install -q pytube
!pip install -q git+https://github.com/openai/whisper.git

import os, re
import torch
from pathlib import Path
from pytube import YouTube

import whisper
from whisper.utils import get_writer



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 39.4 MB/s eta 0:00:00


In [3]:
# Setting up the device agnostic code

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
# Loading up the whisper model for the audio trainscription

model = whisper.load_model("medium.en").to(device)

100%|█████████████████████████████████████| 1.42G/1.42G [00:26<00:00, 57.4MiB/s]


In [5]:
# Mentioning the youtube video and downloading the audio from the youtube video

video_url = 'https://www.youtube.com/watch?v=Sby1uJ_NFIY'
yt = YouTube(video_url)
video = yt.streams.filter(only_audio=True).first()
filename = "audio.mp4"
video.download(filename=filename)

'/content/audio.mp4'

In [7]:
audio = "audio."
result = model.transcribe("Trippy.mp3")
output_directory = "./"


# Save as an SRT file
srt_writer = get_writer("srt", output_directory)
srt_writer(result, audio)


# Save as a VTT file
vtt_writer = get_writer("vtt", output_directory)
vtt_writer(result, audio)

# txt_path = file_path.with_suffix(".txt")
# print(f"\nCreating text file")

# with open(txt_path, "w", encoding="utf-8") as txt:
#     txt.write(result["text"])


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [8]:
txt_writer = get_writer("txt", output_directory)
txt_writer(result, audio)

In [9]:
# Open the file in read mode
with open('/content/audio.txt', 'r') as file:
    text = file.read()

# Now you can use the 'text' variable
print(text)

Hi, so I'm not feeling very well today
I was feeling a bit nauseous in the morning and I had a bit of temperature
So what can you suggest me for for the medications?
I have not prescribed to any doctor, but I want some
medications for home remedies



In [ ]:
import re

# Read the VTT file
with open("audio.vtt", "r") as file:
    vtt_content = file.read()

# Extract timestamps and text segments
pattern = r"(\d+:\d+\.\d+) --> (\d+:\d+\.\d+)\n(.+?)\n\n"
matches = re.findall(pattern, vtt_content, re.DOTALL)

# Calculate start and end times for each segment
aligned_segments = []
for match in matches:
    start_time = match[0]
    end_time = match[1]
    text = match[2].strip()

    # Convert timestamp to seconds
    start_seconds = sum(float(x) * 60 ** i for i, x in enumerate(reversed(start_time.split(":"))))
    end_seconds = sum(float(x) * 60 ** i for i, x in enumerate(reversed(end_time.split(":"))))

    # Append the aligned segment
    aligned_segments.append({"start_time": start_seconds, "end_time": end_seconds, "text": text})


In [ ]:
aligned_segments

In [10]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [11]:
from openai import OpenAI
client = OpenAI(api_key="OPENAI_KEY")

response = client.chat.completions.create(
  model="gpt-3.5-turbo-16k",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are a virtual doctor. You respond empathetically and ensure to understand the user's root cause of the problem. You provide safe medicines and do not prescribe any suspicious or unsafe medications. You answer only questions related to healthcare."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"The users problem is :{text}"
        }
      ]
    },
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)


# print("Content: ", response)

In [12]:
content = response.choices[0].message.content
content

"I'm sorry to hear that you're not feeling well. It's important to understand the root cause of your symptoms before recommending any medications. Nausea and mild fever can be caused by various factors such as a viral infection, food poisoning, or even just a common cold.\n\nInstead of prescribing medications, I can suggest some general home remedies that might help you feel better:\n\n1. Stay hydrated: Drink plenty of fluids like water, herbal tea, or clear broth to stay hydrated and prevent further dehydration.\n\n2. Rest: Get enough rest and avoid any strenuous activities until you start feeling better.\n\n3. Ginger: Ginger has been known to ease nausea. You can try sipping on ginger tea or chewing on a small piece of ginger.\n\n4. Avoid heavy meals: Stick to bland, easy-to-digest foods like toast, rice, or bananas until your symptoms subside.\n\nHowever, if your symptoms worsen or persist for more than a few days, it's important to consult a healthcare professional for a proper dia

In [13]:
VOICE_ID = "nxo2TAmShnYtYe9wngSQ"

In [15]:
import requests

CHUNK_SIZE = 1024
url = "https://api.elevenlabs.io/v1/text-to-speech/nxo2TAmShnYtYe9wngSQ"

headers = {
  "Accept": "audio/mpeg",
  "Content-Type": "application/json",
  "xi-api-key": "ELEVEN_LABS_API"
}

data = {
  "text": f"{content}",
  "model_id": "eleven_monolingual_v1",
  "voice_settings": {
    "stability": 0.5,
    "similarity_boost": 0.5
  }
}

response = requests.post(url, json=data, headers=headers)
with open('output.mp3', 'wb') as f:
    for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)
